In [71]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [72]:
# Load API Credentials
with open('/Users/thoma/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [73]:
# set our API call parameters and filename before the first call
LOCATION = 'Toronto'
TERM = 'Sushi'

In [74]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = f"Data/results_Toronto_Sushi.json"
JSON_FILE

'Data/results_Toronto_Sushi.json'

In [75]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_Toronto_Sushi.json already exists.


In [76]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 788 previous results found.


In [77]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [78]:
## How many results total?
total_results = results['total']
total_results

788

In [79]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

0

In [80]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

ZeroDivisionError: division by zero

In [81]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [82]:
# add progress bar
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/40 [00:00<?, ?it/s]

In [83]:
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
#     display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)

  0%|          | 0/40 [00:00<?, ?it/s]

In [84]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,hDy-uY7Vy_TZdGBzw59lhA,saku-sushi-toronto-2,Saku Sushi,https://s3-media3.fl.yelpcdn.com/bphoto/xBtbkh...,False,https://www.yelp.com/biz/saku-sushi-toronto-2?...,514,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 43.6482, 'longitude': -79.4003}",[],$$,"{'address1': '478 Queen Street W', 'address2':...",+14163687258,+1 416-368-7258,2921.275091
1,q9_gLvTNf11etVxbH7JY0Q,yasu-toronto-8,Yasu,https://s3-media3.fl.yelpcdn.com/bphoto/X7fGAd...,False,https://www.yelp.com/biz/yasu-toronto-8?adjust...,397,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 43.66269, 'longitude': -79.40314}",[],$$$$,"{'address1': '81 Harbord Street', 'address2': ...",+14164772361,+1 416-477-2361,1382.224247
2,RtUvSWO_UZ8V3Wpj0n077w,kinka-izakaya-original-toronto-4,KINKA IZAKAYA ORIGINAL,https://s3-media3.fl.yelpcdn.com/bphoto/nHFVve...,False,https://www.yelp.com/biz/kinka-izakaya-origina...,1721,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 43.66042406856627, 'longitude': -...",[restaurant_reservation],$$,"{'address1': '398 Church Street', 'address2': ...",+14169770999,+1 416-977-0999,2109.959512
3,G1Ce_QBHwT_R8Fc2-ARrwQ,tachi-toronto-2,Tachi,https://s3-media3.fl.yelpcdn.com/bphoto/kLNyCz...,False,https://www.yelp.com/biz/tachi-toronto-2?adjus...,162,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 43.6502911, 'longitude': -79.3839...",[],$$$$,"{'address1': '111 Richmond Street W', 'address...",+14379746367,+1 437-974-6367,2861.455410
4,CCiSthI4jR1EdKg1HJMcNw,yuugi-izakaya-toronto,Yuugi Izakaya,https://s3-media2.fl.yelpcdn.com/bphoto/-sUhJG...,False,https://www.yelp.com/biz/yuugi-izakaya-toronto...,67,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 43.65574, 'longitude': -79.39382}",[],NaN,"{'address1': '49 Baldwin Street', 'address2': ...",+16473438282,+1 647-343-8282,2070.227588


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
783,anGYRueZJKYCJefKFccfjA,made-in-china-hot-pot-toronto-2,Made In China Hot Pot,https://s3-media2.fl.yelpcdn.com/bphoto/SDuYa8...,False,https://www.yelp.com/biz/made-in-china-hot-pot...,70,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",3.5,"{'latitude': 43.8061957242859, 'longitude': -7...",[],$$,"{'address1': '3280 Midland Ave E', 'address2':...",+14163328807,+1 416-332-8807,17055.490481
784,7hUp4XxmUCGqvPFAM8IJww,moxies-scarborough,Moxies,https://s3-media2.fl.yelpcdn.com/bphoto/hZa74e...,False,https://www.yelp.com/biz/moxies-scarborough?ad...,113,"[{'alias': 'newcanadian', 'title': 'Canadian (...",3.0,"{'latitude': 43.7754047, 'longitude': -79.2541...",[],$$,"{'address1': '215 300 Borough Drive', 'address...",+14162968803,+1 416-296-8803,15945.689336
785,fHeaQtfx7HuPzXKeO1XwwQ,moxies-vaughan-2,Moxies,https://s3-media2.fl.yelpcdn.com/bphoto/KZYjwX...,False,https://www.yelp.com/biz/moxies-vaughan-2?adju...,83,"[{'alias': 'newcanadian', 'title': 'Canadian (...",3.5,"{'latitude': 43.7871508901057, 'longitude': -7...",[],$$,"{'address1': '30 Colossus Dr', 'address2': 'Un...",+19052658090,+1 905-265-8090,17211.567259
786,zYPc6o81Em23wRW8oq7_3w,real-canadian-superstore-scarborough-2,Real Canadian Superstore,https://s3-media2.fl.yelpcdn.com/bphoto/1kNz7_...,False,https://www.yelp.com/biz/real-canadian-superst...,36,"[{'alias': 'grocery', 'title': 'Grocery'}]",3.0,"{'latitude': 43.77815442534, 'longitude': -79....",[],$$,"{'address1': '1755 Brimley Road', 'address2': ...",+14162790802,+1 416-279-0802,15765.415170
787,SYExIal3fh3mS3kpoP9Ixg,eds-real-scoop-toronto,Ed's Real Scoop,https://s3-media4.fl.yelpcdn.com/bphoto/amdv9-...,False,https://www.yelp.com/biz/eds-real-scoop-toront...,121,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",4.5,"{'latitude': 43.67278, 'longitude': -79.2879899}",[],$$,"{'address1': '2224 Queen St E', 'address2': ''...",+14166996100,+1 416-699-6100,8752.348908


In [85]:
# check for duplicate IDs
final_df.duplicated(subset='id').sum()

0

In [86]:
# save the final results to a compressed csv
final_df.to_csv('Data/results_Toronto_Sushi.csv.gz', compression='gzip',index=False)